# Big Data Project
## Outcome: Divergence

In this notebook we will outcome 'Divergence'

'Divergence' will be used to detect matches with a big difference between bet houses quotas.


In [1]:
from pyspark.sql import SparkSession
import pyspark

spark = SparkSession \
    .builder \
    .appName("Bets Exploring") \
    .getOrCreate()
    
sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=Bets Exploring>

In [2]:
pathori = "../Data/Raw/Main/"
pathtemp = "../Data/Interim/"
pathdest = "../Data/Processed/"
filetemp = "main_competitions.csv"
filedest = "main_competitions.csv"
bethouses = ['WH','SB','IW','GB']
quotatypes = ['H','D','A']

In [3]:
# Open dataset
df = spark.read.csv(path = pathdest + filedest, header = True, inferSchema = True)
df = df.withColumn("Date", df["Date"].cast("date"))

In [4]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Div: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTR: string (nullable = true)
 |-- WHH: double (nullable = true)
 |-- WHD: double (nullable = true)
 |-- WHA: double (nullable = true)
 |-- SBH: double (nullable = true)
 |-- SBD: double (nullable = true)
 |-- SBA: double (nullable = true)
 |-- IWH: double (nullable = true)
 |-- IWD: double (nullable = true)
 |-- IWA: double (nullable = true)
 |-- GBH: double (nullable = true)
 |-- GBD: double (nullable = true)
 |-- GBA: double (nullable = true)



In [5]:
df.show(10)

+-------+-------------+---------+---+----------+--------------+-------------+---+----+----+----+----+----+----+----+---+---+----+----+----+
|Country|  Competition|   Season|Div|      Date|      HomeTeam|     AwayTeam|FTR| WHH| WHD| WHA| SBH| SBD| SBA| IWH|IWD|IWA| GBH| GBD| GBA|
+-------+-------------+---------+---+----------+--------------+-------------+---+----+----+----+----+----+----+----+---+---+----+----+----+
|Belgium|JupilerLeague|2003-2004| B1|2003-08-08|   Club Brugge|         Genk|  H|null|null|null|1.44|3.75| 6.5|1.45|3.8|5.4| 1.4| 3.8|6.85|
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08|    Anderlecht|      Antwerp|  H|1.22| 5.0| 9.5| 1.2| 5.5|10.0| 1.3|4.2|8.0| 1.2| 5.5|9.25|
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08|     Charleroi|       Lierse|  A| 2.5| 3.4| 2.3|2.62| 3.4|2.25| 2.9|3.2|2.1| 2.5|3.35|2.35|
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08|      Germinal|       Bergen|  H| 1.9| 3.5| 3.2| 1.9| 3.4| 3.4| 1.9|3.4|3.2|1.85|3.35| 3.5|
|Belgium|JupilerLeag

In [6]:
df.count()

106839

In [7]:
df.describe('Country','Competition','Season','Div','Date','HomeTeam','AwayTeam','FTR').show()

+-------+-------+------------+---------+------+------------+------------+------+
|summary|Country| Competition|   Season|   Div|    HomeTeam|    AwayTeam|   FTR|
+-------+-------+------------+---------+------+------------+------------+------+
|  count| 106839|      106839|   106839|106806|      106569|      106569|106805|
|   mean|   null|        null|     null|  null|        null|        null|  null|
| stddev|   null|        null|     null|  null|        null|        null|  null|
|    min|Belgium|Championship|2003-2004|    B1|A. Sebatspor|A. Sebatspor|     A|
|    max| Turkey|      SerieB|2018-2019|    T1|      Zwolle|      Zwolle|     H|
+-------+-------+------------+---------+------+------------+------------+------+



In [8]:
df.describe('WHH','WHD','WHA','SBH','SBD','SBA','IWH','IWD','IWA','GBH','GBD','GBA').show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|               WHH|               WHD|               WHA|               SBH|               SBD|              SBA|               IWH|               IWD|               IWA|               GBH|               GBD|              GBA|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|            104253|            104253|            104253|             56510|             56511|            56511|            105397|            105397|            105393|             64436|             64436|            64436|
|   mean|2.3983402875697903|3.4690791631896793| 3.937763

In [9]:
dfok = df.dropna(how='any')
print(dfok.count())
dfok.show(5)

53472
+-------+-------------+---------+---+----------+----------+-----------+---+----+---+---+----+---+----+----+---+---+----+----+----+
|Country|  Competition|   Season|Div|      Date|  HomeTeam|   AwayTeam|FTR| WHH|WHD|WHA| SBH|SBD| SBA| IWH|IWD|IWA| GBH| GBD| GBA|
+-------+-------------+---------+---+----------+----------+-----------+---+----+---+---+----+---+----+----+---+---+----+----+----+
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08|Anderlecht|    Antwerp|  H|1.22|5.0|9.5| 1.2|5.5|10.0| 1.3|4.2|8.0| 1.2| 5.5|9.25|
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08| Charleroi|     Lierse|  A| 2.5|3.4|2.3|2.62|3.4|2.25| 2.9|3.2|2.1| 2.5|3.35|2.35|
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08|  Germinal|     Bergen|  H| 1.9|3.5|3.2| 1.9|3.4| 3.4| 1.9|3.4|3.2|1.85|3.35| 3.5|
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08|   Lokeren| St Truiden|  D|1.66|3.6|4.0|1.72|3.5| 4.0| 1.9|3.4|3.2| 1.8| 3.4|3.65|
|Belgium|JupilerLeague|2003-2004| B1|2003-09-08|  Westerlo|Louvieroise|  D|1.

In [28]:
# Divergence calculation simulation

match = dfok.take(1)[0]
print ("Match: ", match)
print ("Bet houses: ", bethouses)

resultList = ['H','D','A']
divergence = 0
for result in resultList:
    print ()
    min = 1000
    max = 0
    sum = 0
    count = 0
    for bethouse in bethouses:
        col = bethouse + result
        val = match[col]
        if not val is None:
            print (col, ':', val, end='  ')
            sum = sum + val
            count = count + 1
            if val > max:
                max = val
            if val < min:
                min = val
    mean = sum/count
    diverMax = (max / mean) - 1
    if diverMax > divergence:
        divergence = diverMax
    diverMin = abs((min / mean) - 1)
    if diverMin > divergence:
        divergence = diverMin
    
    print ('\nCount:', count, end='  ')
    print ('Sum:', sum, end='  ')
    print ('Max:', max, end='  ')
    print ('Min:', min, end='  ')
    print ('Mean:', mean)
    print ('Diver 1:  ', diver1, end='  ')
    print ('Diver 2:  ', diver2, end='  ')
    print ('Divergence: ', divergence)
       

Match:  Row(Country='Belgium', Competition='JupilerLeague', Season='2003-2004', Div='B1', Date=datetime.date(2003, 9, 8), HomeTeam='Anderlecht', AwayTeam='Antwerp', FTR='H', WHH=1.22, WHD=5.0, WHA=9.5, SBH=1.2, SBD=5.5, SBA=10.0, IWH=1.3, IWD=4.2, IWA=8.0, GBH=1.2, GBD=5.5, GBA=9.25)
Bet houses:  ['WH', 'SB', 'IW', 'GB']

WHH : 1.22  SBH : 1.2  IWH : 1.3  GBH : 1.2  
Count: 4  Sum: 4.92  Max: 1.3  Min: 1.2  Mean: 1.23
Diver 1:   0.08843537414965996  Diver 2:   0.12925170068027214  Divergence:  0.05691056910569103

WHD : 5.0  SBD : 5.5  IWD : 4.2  GBD : 5.5  
Count: 4  Sum: 20.2  Max: 5.5  Min: 4.2  Mean: 5.05
Diver 1:   0.08843537414965996  Diver 2:   0.12925170068027214  Divergence:  0.16831683168316824

WHA : 9.5  SBA : 10.0  IWA : 8.0  GBA : 9.25  
Count: 4  Sum: 36.75  Max: 10.0  Min: 8.0  Mean: 9.1875
Diver 1:   0.08843537414965996  Diver 2:   0.12925170068027214  Divergence:  0.16831683168316824


In [29]:
def calcDivergence(match, bethouses) :
    resultList = ['H','D','A']
    divergence = 0
    for result in resultList:
        min = 1000
        max = 0
        sum = 0
        count = 0
        for bethouse in bethouses:
            col = bethouse + result
            val = match[col]
            if not val is None:
                sum = sum + val
                count = count + 1
                if val > max:
                    max = val
                if val < min:
                    min = val
        mean = sum/count
        diverMax = (max / mean) - 1
        if diverMax > divergence:
            divergence = diverMax
        diverMin = abs((min / mean) - 1)
        if diverMin > divergence:
            divergence = diverMin
            
    return divergence

In [30]:
match = dfok.take(1)[0]
divergence = calcDivergence(match, bethouses)
print (match)
print (divergence)

Row(Country='Belgium', Competition='JupilerLeague', Season='2003-2004', Div='B1', Date=datetime.date(2003, 9, 8), HomeTeam='Anderlecht', AwayTeam='Antwerp', FTR='H', WHH=1.22, WHD=5.0, WHA=9.5, SBH=1.2, SBD=5.5, SBA=10.0, IWH=1.3, IWD=4.2, IWA=8.0, GBH=1.2, GBD=5.5, GBA=9.25)
0.16831683168316824


In [32]:
rdddiver = dfok.rdd.map(lambda match: (match, calcDivergence(match, bethouses)))
rdddiver.take(5)

[(Row(Country='Belgium', Competition='JupilerLeague', Season='2003-2004', Div='B1', Date=datetime.date(2003, 9, 8), HomeTeam='Anderlecht', AwayTeam='Antwerp', FTR='H', WHH=1.22, WHD=5.0, WHA=9.5, SBH=1.2, SBD=5.5, SBA=10.0, IWH=1.3, IWD=4.2, IWA=8.0, GBH=1.2, GBD=5.5, GBA=9.25),
  0.16831683168316824),
 (Row(Country='Belgium', Competition='JupilerLeague', Season='2003-2004', Div='B1', Date=datetime.date(2003, 9, 8), HomeTeam='Charleroi', AwayTeam='Lierse', FTR='A', WHH=2.5, WHD=3.4, WHA=2.3, SBH=2.62, SBD=3.4, SBA=2.25, IWH=2.9, IWD=3.2, IWA=2.1, GBH=2.5, GBD=3.35, GBA=2.35),
  0.102661596958175),
 (Row(Country='Belgium', Competition='JupilerLeague', Season='2003-2004', Div='B1', Date=datetime.date(2003, 9, 8), HomeTeam='Germinal', AwayTeam='Bergen', FTR='H', WHH=1.9, WHD=3.5, WHA=3.2, SBH=1.9, SBD=3.4, SBA=3.4, IWH=1.9, IWD=3.4, IWA=3.2, GBH=1.85, GBD=3.35, GBA=3.5),
  0.05263157894736836),
 (Row(Country='Belgium', Competition='JupilerLeague', Season='2003-2004', Div='B1', Date=dateti

In [33]:
dfdiver = dfok.withColumn("Divergence", calcDivergence(dfok, bethouses))

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [ ]:
dfspain.orderBy('WHA', ascending=False).show(5)


In [ ]:
dfspain = dfspain.withColumn('mean', (dfspain['WHA']+dfspain['IWA'])/2)


In [ ]:
dfspain = dfspain.withColumn('mean', (dfspain['WHA']+dfspain['IWA'])/2)
dfspain.show(10)